### ИНФО

Имеется выгрузка с временной статистикой работы асессоров над однотипным заданием.Задание может состоять из одного или несколько микрозаданий. Время резервирования задания (assigned_ts) указывает на тот момент, когда система назначила определенного асессора исполнителем этого задания. Этот момент может совпадать с временем начала работы асессора над заданием, а может и не совпадать (асессор может отойти выпить чаю, а потом приступить к заданию, асессор может выполнять предыдущее задание, в то время как за ним зарезервированы новые).
Предположим, что асессор за 30 секунд своего рабочего времени получает N рублей.
Какую оплату вы считаете справедливой для выполнения асессором одного микрозадания из этого файла? Опишите подробно все этапы вашего решения.


Формат исходника: 
 - login — логин асессора
 - tid — id оцениваемого задания (task id)
 - Microtasks – количество микрозаданий в одном задании
 - assigned_ts — время резервирования системой задания для асессора
 - closed_ts — точное время завершения работы над заданием
 
 разделитель — табуляция \t

### ИМПОРТ ИНСТРУМЕНТОВ

In [18]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import datetime as dt 
from scipy.stats import t
import math

### ОЗНАКОМЛЕНИЕ С ДАННЫМИ

In [19]:
#читаем файл с предустановкой форматов для каждого из столбцов и распознованием дат
dtypes={'login': 'str', 'tid': 'int', 'Microtasks': 'int', 'assigned_ts': 'str', 'closed_ts':'str'}
parse_dates=['assigned_ts','closed_ts']
db=pd.read_csv(r'C:\Users\devba\Documents\DataScience\Vacancies\файл 1.txt',sep='\t', dtype=dtypes, parse_dates=parse_dates)

In [20]:
#так выглядят записи нашей таблицы
db.sample(10)

,login,tid,Microtasks,assigned_ts,closed_ts
16194,login7,195084352,1,2017-05-12 02:33:44,2017-05-12 02:36:27
131607,login132,190734192,4,2017-04-21 19:03:57,2017-04-21 19:06:16
61452,login62,194236860,1,2017-05-07 15:16:26,2017-05-07 15:31:45
697428,login755,170726870,6,2017-05-25 23:59:49,2017-05-26 00:04:15
383749,login418,196789282,7,2017-05-21 21:53:43,2017-05-21 21:55:49
408036,login438,197757984,1,2017-05-27 11:32:54,2017-05-27 11:33:04
637768,login699,190593520,2,2017-04-20 14:12:15,2017-04-20 14:16:11
192838,login179,193753592,1,2017-05-04 13:38:50,2017-05-04 13:40:31
351075,login373,195038968,2,2017-05-11 22:46:01,2017-05-11 22:46:44
64881,login65,171134330,3,2017-05-30 20:20:07,2017-05-31 09:46:56


In [5]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701827 entries, 0 to 701826
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   login        701827 non-null  object        
 1   tid          701827 non-null  int32         
 2   Microtasks   701827 non-null  int32         
 3   assigned_ts  701827 non-null  datetime64[ns]
 4   closed_ts    701827 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int32(2), object(1)
memory usage: 21.4+ MB


пропусков в данных нет , всего в таблице 701827 строк 

### ГЕНЕРАЦИЯ НОВЫХ ДАННЫХ

In [22]:
#введем новый столбец, показывающий время, затраченное на выполнение каждого задания
db['time_per_task']=(db.closed_ts-db.assigned_ts).dt.seconds

In [23]:
#введем новый столбец, показывающий время, затраченное на выполнение каждого микрозадания
db['av_time_per_microtask']=np.round(db.time_per_task/db.Microtasks,2)

In [8]:
db.sample(10)

,login,tid,Microtasks,assigned_ts,closed_ts,time_per_task,av_time_per_microtask
545857,login595,194960462,1,2017-05-11 13:26:43,2017-05-12 08:24:33,68270,68270.00
656628,login718,190565212,1,2017-04-20 18:25:48,2017-04-20 18:26:16,28,28.00
331930,login345,191327804,1,2017-04-23 14:30:30,2017-04-23 14:35:15,285,285.00
633676,login698,184144100,2,2017-04-27 10:10:38,2017-04-27 10:14:03,205,102.50
420585,login464,192519516,7,2017-04-28 12:03:39,2017-04-28 12:14:24,645,92.14
257712,login250,195509666,1,2017-05-15 09:56:15,2017-05-15 09:56:32,17,17.00
683569,login745,191097268,2,2017-04-22 15:15:57,2017-04-22 15:17:07,70,35.00
255904,login250,190724728,2,2017-04-22 03:45:57,2017-04-22 03:49:14,197,98.50
166667,login153,171642326,6,2017-05-21 14:43:51,2017-05-21 15:02:30,1119,186.50
22724,login7,198294027,1,2017-05-31 12:05:38,2017-05-31 12:23:04,1046,1046.00


В вводной информации было отмечено, что: 
    1 все задания однотипны 
    2 время назначения задания асессору и начало его работы над заданием могут не совпадать

Делаем вывод, что если однотипны задания, то однотипны и микрозадания. Следовательно, в случаях, когда асессор приступил к выполнению задания не сразу длительность выполнения им конкретного задания будет велика в сравнении со средней затратой по времени выполнения однотипных заданий. Вычислить эти отклонения поможет поиск выбросов.

### ОЧИСТКА ДАННЫХ

In [24]:
#функция по отбрасыванию статистических выбросов 
def outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3-Q1
    l_boundary=Q1-1.5*IQR
    h_boundary=Q3+1.5*IQR
    
    outliers = column[~column.between(l_boundary, h_boundary)]
    n_outl=outliers.count()
    p_outl=np.round(n_outl*100/len(db),2)
    print('Minor boundries are ({},{})\n There are {} or {}% outlying values'\
          .format(l_boundary,h_boundary,n_outl,p_outl))
    
    inp=input('Clear outliers?\n Input Y (for yes) or N (for no): ')
    
    if inp=='Y': 
        if (type(l_boundary)==int)|(type(l_boundary)==np.float64):
            column=column[column.between(l_boundary, h_boundary)]
        else: 
            column=column[column.between(str(l_boundary), str(h_boundary))]
    else: 
        column=column
    
    return column

In [13]:
cleaned_db=outliers(db['av_time_per_microtask']).describe()
cleaned_db

Minor boundries are (-180.18500000000003,398.975)
 There are 94394 or 13.45% outlying values
Clear outliers?
 Input Y (for yes) or N (for no): Y


count    607433.000000
mean         90.875643
std          81.649168
min           0.220000
25%          33.000000
50%          62.000000
75%         119.000000
max         398.880000
Name: av_time_per_microtask, dtype: float64

### АНАЛИЗ И УМОЗАКЛЮЧЕНИЕ

Доля выбросов в общей таблице сравнительно невелика: 13,45%, поэтому от них можно избавиться  для получения данных больше соответствующих реальности. На основании уже очищенных данных мы можем посчитать доверительный интервал затрат по времени на выполнение одного микрозадания с точностью 99%. 

In [25]:
def confidence_interval_t(alpha, s, n, mean):
    value = -t.ppf(alpha / 2, n - 1) * s / math.sqrt(n)
    return mean - value, mean + value

In [26]:
alpha=0.005
s=cleaned_db[2]
n=cleaned_db[0]
mean=cleaned_db[5]
confidence_interval_t(alpha, s, n, mean)

(61.70592927275494, 62.29407072724506)

Полученный результат указывает на то, что в 99% случаев на выполнение одного микрозадания в среднем сазтрачивается от 61,70 до 62,29 секунд. Нам известно, что за 30 секунд рабочего времени асессор получает N руб. Делим средние затраты на одно микрозадание на 30 секунд.

In [201]:
l_b=61.71/30
h_b=62.29/30
print(round(l_b,2), round(h_b,2))

2.06 2.08


Получается, что справедливая оплата за выполнение одного микрозадания должна лежать в интервале [2,06N, 2,08N] руб.